# APIM ❤️ OpenAI

## Semantic Caching lab
![flow](../../images/semantic-caching.gif)

Playground to try the [sementic caching policy](https://learn.microsoft.com/en-us/azure/api-management/azure-openai-semantic-cache-lookup-policy). 

The azure-openai-semantic-cache-lookup policy conducts a cache lookup of responses on Azure OpenAI Chat Completion API and Completion API requests from a pre-configured external cache. It operates by comparing the vector proximity of the prompt to prior requests and using a specific similarity score threshold. Caching responses helps reduce bandwidth and processing demands on the backend Azure OpenAI API, thus reducing latency perceived by API consumers.

### Result
![result](result.png)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [1]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"icsu-aigw-dev" # change the name to match your naming style
resource_group_location = "southeastasia"
apim_resource_name = "icsu-apim"
apim_resource_location = "southeastasia"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "icsuaoaijpe", "location": "japaneast"}, {"name": "icsuaoaiaue", "location": "australiaeast"}  ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_lb_config = [ {"name": "icsuaoaijpe", "priority": 1, "weight": 100}, {"name": "icsuaoaiaue", "priority": 2, "weight": 300}  ] # advanced load balancer configuration for OpenAI resources that will be stored as a named value
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "icsuaoaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "icsuaoaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]
log_analytics_name = "icsuws"
app_insights_name = 'icsuai'
embeddings_model_name = "text-embedding-ada-002"
embeddings_model_version = "2"
embeddings_deployment_name = "text-embedding-ada-002"
rediscache_name = "icsuredis"
app_registration_name = "icsu-aigw-app"



### 1️⃣ Create the App Registration in Microsoft Entra ID
The following command creates a client application registration

In [2]:
cmd_stdout = ! az account show --query homeTenantId --output tsv
tenant_id = cmd_stdout.n

cmd_stdout = ! az ad app create --display-name {app_registration_name} --query appId --output tsv
client_id = cmd_stdout.n


### 2️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [12]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Grpup ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Grpup  icsu-aigw-dev  created ⌚  14:09:57.622242


### 3️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [13]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id).replace("{aad-client-application-id}", client_id).replace("{aad-tenant-id}", tenant_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "embeddingsModelName": { "value": embeddings_model_name },
    "embeddingsModelVersion": { "value": embeddings_model_version },
    "embeddingsDeploymentName": { "value": embeddings_deployment_name },
    "redisCacheName": { "value": rediscache_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


/home/ythuang/workspace/AI-Gateway/labs/all-in-one/main.bicep(368,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is an inaccuracy in the documentation, please report it to the Bicep Team. [https://aka.ms/bicep-type-issues]
/home/ythuang/workspace/AI-Gateway/labs/all-in-one/main.bicep(385,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is an inaccuracy in the documentation, please report it to the Bicep Team. [https://aka.ms/bicep-type-issues]

{\ Finished .. - Starting ..
  "id": "/subscriptions/513df66c-64b0-4c0b-a13a-7f37bb384aff/resourceGroups/icsu-aigw-dev/providers/Microsoft.Resources/deployments/all-in-one",
  "location": null,
  "name": "all-in-one",
  "properties": {
    "correlationId": "322cc029-6c35-43ee-8d22-f0af4aac08c3",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
 

1185

### 4️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [14]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = deployment_stdout.n
print("👉🏻 APIM Service Id: ", apim_service_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



👉🏻 APIM Service Id:  /subscriptions/513df66c-64b0-4c0b-a13a-7f37bb384aff/resourceGroups/icsu-aigw-dev/providers/Microsoft.ApiManagement/service/icsu-apim-d2fighzpcjauk
👉🏻 API Gateway URL:  https://icsu-apim-d2fighzpcjauk.azure-api.net
👉🏻 Workspace ID:  82e54f3a-5cdd-4375-b30e-4216f16c90a3
👉🏻 App ID:  15336def-8dfe-420f-9086-3d94c22f8b0a


### 5️⃣ Create a device flow to get the access token

In [9]:
import json
import logging

import requests
import msal

app = msal.PublicClientApplication(
    client_id, authority="https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes=["User.Read"])
if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))

print(flow["message"])



To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FT8T3S9E8 to authenticate.


### 6️⃣ Acquire the token and query the graph API


In [10]:
result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result['access_token']
    # Calling graph using the access token
    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent=2))
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug

Graph API call result: {
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",
  "businessPhones": [],
  "displayName": "Yingting Huang",
  "givenName": "Yingting",
  "jobTitle": null,
  "mail": "ythuang@microsoft.com",
  "mobilePhone": null,
  "officeLocation": null,
  "preferredLanguage": null,
  "surname": "Huang",
  "userPrincipalName": "ythuang_microsoft.com#EXT#@MngEnv105102.onmicrosoft.com",
  "id": "0e3e7aa3-0183-483f-9eee-ac4705cc6ed4"
}


### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [ ]:
import time
runs = 0
sleep_time_ms = 1000

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
    start_time = time.time()
    response = requests.post(url, headers = {'api-key':apim_subscription_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
    response_time = time.time() - start_time
    print(f"response time: {response_time:.2f} seconds")
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)

### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [ ]:
from openai import AzureOpenAI
import time
import random

runs = 20
sleep_time_ms = 0
questions = ["Can you tell me the time, please?", "Would you be so kind as to inform me of the current time, if possible?", "Could you please inform me of the current time?", "Could you kindly inform me of the current time, please?"]
api_runs = []  # Response Times for each run

for i in range(runs):
    print("▶️ Run: ", i+1)
    random_question = random.choice(questions)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": random_question}
    ]
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    start_time = time.time()
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
    response_time = time.time() - start_time
    print("Q:", random_question)
    print(f"t: {response_time:.2f} seconds")
    api_runs.append(response_time)
    print("R:", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


### 🔍 Analyze Semantic Caching performance


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 5]
df = pd.DataFrame(api_runs, columns=['Response Time'])
df['Run'] = range(1, len(df) + 1)
df.plot(kind='bar', x='Run', y='Response Time', legend=False)
plt.title('Semantic Caching Performance')
plt.xlabel('Runs')
plt.ylabel('Response Time (s)')
plt.xticks(df['Run'], rotation=0)  # Set x-axis ticks to be the run numbers

average = df['Response Time'].mean()
plt.axhline(y=average, color='r', linestyle='--', label=f'Average: {average:.2f}')
plt.legend()

plt.show()

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.